In [1]:
import django_initializer
from telemetry.models import FastLap, Lap
from telemetry.analyzer import Analyzer
from telemetry.influx import Influx
import plotly.io as pio

# pio.renderers.default = "svg"  # comment this line to use interactive plots

import pandas as pd
from telemetry.visualizer import *

%load_ext autoreload
%autoreload 2

analyzer = Analyzer()
influx = Influx()

2023-06-04 15:52:02,789 DEBUG Influx: Connected to https://telemetry.b4mad.racing/


In [2]:
# Automobilista 2 - SpaFrancorchamps:Spa_Francorchamps_2022 - Mercedes-Benz CLK LM
# iRacing / richmond / Gen 4 Chevrolet Impala
kwargs = {
    "game__name": "iRacing",
    # "track__name": "fuji nochicane",
    # "track__name": "longbeach",
    "track__name": "oschersleben gp",
    # "track__name": "oran gp",
    # "track__name": "nurburgring combined",
    # "track__name": "richmond",
    "car__name": "Ferrari 488 GT3 Evo 2020",
    # "car__name": "Gen 4 Chevrolet Impala",
}
# kwargs = {
#     "game__name": "Automobilista 2",
#     # "track__name": "SpaFrancorchamps:Spa_Francorchamps_2022",
#     "track__name": "Nurburgring_2020:Nordschleife_2020",
#     "car__name": "Porsche 911 RSR GTE",
#     # "car__name": "Mercedes-Benz CLK LM",
# }
lap_id = 0
# lap_id = 43525

laps = []
if lap_id:
    lap = Lap.objects.get(pk=lap_id)
    laps.append(lap)
else:
    fast_laps = FastLap.objects.filter(**kwargs)
    laps.append(fast_laps.first().laps.first())

data_frames = []
for lap in laps:
    print(lap)
    lap_df = influx.telemetry_for_laps([lap], measurement="fast_laps", bucket="fast_laps")[0]
    data_frames.append(
        {
            "lap": lap,
            "df": lap_df,
        }
    )

2023-06-04 15:52:04,303 INFO Processing iRacing - oschersleben gp - Ferrari 488 GT3 Evo 2020
2023-06-04 15:52:04,304 INFO   track.id 1040 car.id 9
2023-06-04 15:52:04,304 INFO   session 1685815222 lap.id 113864 number 3
2023-06-04 15:52:04,304 INFO   length 3609 time 85.3591 valid True
2023-06-04 15:52:04,304 INFO   start 2023-06-03 20:03:22.798362+00:00 end 2023-06-03 20:04:48.126951+00:00


3: 20:03:22 - 20:04:48 85.3591s 3609m valid: True


In [3]:
for data_frame in data_frames:
    lap = data_frame["lap"]
    lap_df = data_frame["df"]
    # display(lap_df.head())
    columns = ["Brake", "SpeedMs", "Throttle", "Gear", "CurrentLapTime", "SteeringAngle"]
    lap_df = analyzer.resample(lap_df, columns=columns)
    data_frame["df"] = lap_df

    # window_length = 9  # should be odd and >= polynomial_order + 2
    # polynomial_order = 2
    # lap_df['Throttle'] = scipy.signal.savgol_filter(lap_df['Throttle'], window_length, polynomial_order)

    fig = lap_fig(lap_df)
    fig.update_layout(title=dict(text=f"{lap}"))
    fig.show()

    fig = lap_fig(lap_df, columns=["SpeedMs"])
    fig.show()
    fig = lap_fig(lap_df, columns=["Gear"])
    fig.show()
    fig = lap_fig(lap_df, columns=["CurrentLapTime"])
    fig.show()
    fig.update_layout(title=dict(text=f"{lap}"))

In [4]:
track_df = data_frames[0]["df"].copy()

In [5]:
threshold = track_df["Throttle"].max() * 0.98
display(threshold)
sectors = analyzer.split_sectors(track_df, threshold=threshold, min_length=50)
# for i in range(len(sectors)):
#     display(sectors[i].head())
#     fig = lap_fig(sectors[i])
#     fig.update_layout(title=dict(text=f"{i}"))
#     fig.show()
len(sectors)

0.98

9

In [6]:
sector_start_end = analyzer.extract_sector_start_end(sectors, min_length=50)
print(len(sector_start_end))
sector_start_end

9


[{'start': 309, 'end': 869, 'length': 560},
 {'start': 870, 'end': 1339, 'length': 469},
 {'start': 1340, 'end': 1617, 'length': 277},
 {'start': 1618, 'end': 1924, 'length': 306},
 {'start': 1925, 'end': 2596, 'length': 671},
 {'start': 2597, 'end': 2791, 'length': 194},
 {'start': 2792, 'end': 2997, 'length': 205},
 {'start': 2998, 'end': 3182, 'length': 184},
 {'start': 3183, 'end': 308, 'length': 734}]

In [7]:
# for i in (0, -1):
#     sector = analyzer.section_df(track_df, sector_start_end[i]['start'], sector_start_end[i]['end'])
#     fig = lap_fig(sector, mode="markers")
#     fig.update_layout(title=dict(text=f"{i}"))
#     fig.show()

In [8]:
brake_feature_args = {
    "brake_threshold": 0.1,
}
throttle_features_args = {
    # "throttle_threshold": 0.98,
}
sector_dfs = []


for i in range(len(sector_start_end)):
    sector = analyzer.section_df(track_df, sector_start_end[i]["start"], sector_start_end[i]["end"])
    sector_dfs.append(sector)
    fig = lap_fig(sector)
    # lap_fig(sector, columns=["SteeringAngle"], fig=fig)

    throttle_or_brake = analyzer.sector_type(sector)
    # if throttle_or_brake == "brake":
    #     continue
    # if i != 2:
    #     continue
    # fig.show()
    title = f"{i} - {throttle_or_brake}<br>"

    brake_features = analyzer.brake_features(sector, **brake_feature_args)
    title += f"brake: {brake_features}<br>"
    if brake_features:  # and throttle_or_brake == "brake":
        fig_add_features(fig, brake_features)

    throttle_features = analyzer.throttle_features(sector, **throttle_features_args)
    if throttle_features:  # and throttle_or_brake == "throttle":
        fig_add_features(fig, throttle_features, color="green")
    title += f"throttle: {throttle_features}"

    fig.update_layout(title=dict(text=title, font=dict(size=8)))

    fig.show()

In [9]:
monotonic = track_df["CurrentLapTime"].is_monotonic_increasing
if monotonic:
    print("monotonic")
    lookup = analyzer.distance_speed_lookup_table_non_lin(track_df)
else:
    print("not monotonic")
    lookup = analyzer.distance_speed_lookup_table_lin(track_df)

# lookup = analyzer.resample(lookup, columns=["SpeedMs", "CurrentLapTime"])
fig = lap_fig(lookup, columns=["CurrentLapTime", "SpeedMs"])
lap_fig(track_df, columns=["CurrentLapTime"], fig=fig)
fig.show()

lookup
# track_df

not monotonic


,DistanceRoundTrack,CurrentLapTime,SpeedMs
0,3.3,0.000,54.310910
1,4.3,0.025,54.393890
2,5.3,0.049,54.393890
3,6.3,0.074,54.433098
4,7.3,0.099,54.433098
...,...,...,...
3604,3605.2,88.849,53.917816
3605,3606.2,88.874,53.917816
3606,3607.2,88.898,54.009724
3607,3608.2,88.923,54.009724
